In [179]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=100000, n_features=14, n_informative=10, noise=15, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)
X.columns = [f'col_{col}' for col in X.columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [180]:
class MyLineReg:    
    """
    Линейная регрессия

    Parameters
    ----------
    n_iter : int, optional
        Количество шагов градиентного спуска, by default 100
    learning_rate : float, optional
        Коэффициент скорости обучения градиентного спуска.
        Если на вход пришла lambda-функция, то learning_rate вычисляется на каждом шаге на основе переданной функцией
    weights : np.ndarray, optional
        Веса модели
    metric : str, optional
        Метрика, которая будет вычисляться параллельно с функцией потерь.
        Принимает одно из следующих значений: mae, mse, rmse, mape, r2
    reg : str, optional
        Вид регуляризации. Принимает одно из следующих значений: l1, l2, elasticnet, by default None
    l1_coef : float, optional
        Коэффициент L1 регуляризации. Принимает значения от 0.0 до 1.0, by default 0
    l2_coef : float, optional
        Коэффициент L2 регуляризации. Принимает значения от 0.0 до 1.0, by default 0
    sgd_sample : Union[int, float], optional
        Количество образцов, которое будет использоваться на каждой итерации обучения.
        Может принимать целые числа, либо дробные от 0.0 до 1.0, by default None
    random_state : int, optional
        Сид для воспроизводимости результата, by default 42
    """
    
    def __init__(self, 
                 n_iter: int = 100, 
                 learning_rate: float = 0.1,             
                 metric: str | None = None, 
                 reg: str | None = None, 
                 l1_coef: float = 0.0, 
                 l2_coef: float = 0.0, 
                 random_state: int = 42, 
                 sgd_sample: str | None = None) -> None:
                
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = None
        self.metric = metric
        self.best_score = None
        self.metrics = {
            'mae':  lambda y, y_pred: np.mean(np.abs(y - y_pred)),
            'mse':  lambda y, y_pred: np.mean((y - y_pred) ** 2),
            'rmse': lambda y, y_pred: np.sqrt(np.mean((y - y_pred) ** 2)),
            'mape': lambda y, y_pred: np.mean(np.abs((y - y_pred) / y)) * 100,
            'r2':   lambda y, y_pred: 1 - np.sum((y - y_pred) ** 2) / np.sum((y - np.mean(y)) ** 2)
        }
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.sgd_sample = sgd_sample 
        self.random_state = random_state

    def fit(self, X: pd.DataFrame, y: pd.Series, verbose: int | bool = False) -> None:        
        """Обучение линейной регрессии

        Parameters
        ----------
        X : pd.DataFrame
            Все фичи
        y : pd.Series
            Целевая переменная
        verbose : int, optional
            Указывает через сколько итераций градиентного спуска будет выводиться лог
        """
        
        random.seed(self.random_state)
        n_samples, n_features = X.shape
        X = X.copy()
        ones = np.ones(n_samples)
        X.insert(0, 'x_0', ones)
        
        self.weights = np.ones(n_features + 1)

        for iter in range(self.n_iter):
            
            if isinstance(self.sgd_sample, float):
                sample_rows_idx = random.sample(range(X.shape[0]), round(X.shape[0] * self.sgd_sample))                
            if isinstance(self.sgd_sample, int):
                sample_rows_idx = random.sample(range(X.shape[0]), self.sgd_sample)            
            if not self.sgd_sample:
                sample_rows_idx = range(X.shape[0])                                                
            X_batch = X.iloc[sample_rows_idx]
            y_batch = y.iloc[sample_rows_idx]
            
            y_pred = X @ self.weights
            loss = np.sum((y - y_pred) ** 2) / n_samples            
            
            if self.reg == 'l1' or self.reg == 'elasticnet':
                loss += self.l1_coef * np.sum(np.abs(self.weights))
            if self.reg == 'l2' or self.reg == 'elasticnet':
                loss += self.l2_coef * np.sum(self.weights ** 2)
                
            if not isinstance(self.learning_rate, (float, int)):
                lr = self.learning_rate(iter + 1)
                self.weights -= lr * self.__calc_grad(X_batch, y_batch)
            else:
                self.weights -= self.learning_rate * self.__calc_grad(X_batch, y_batch)

            if self.metric:
                self.best_score = self.metrics[self.metric](y, y_pred)

            if verbose and iter % verbose == 0:
                print(f"{iter if iter != 0 else 'start'} | loss: {loss}", f"| {self.metric}: {self.best_score}" if self.metric else '')
    
    def predict(self, X: pd.DataFrame) -> pd.Series:
        """Выдача предсказаний моделью

        Parameters
        ----------
        X : pd.DataFrame
            Матрица фичей
        """
        
        X = X.copy()
        ones = np.ones(X.shape[0])
        X.insert(0, 'x_0', ones)
        return X @ self.weights
    
    def __calc_grad(self, X: pd.DataFrame, y: pd.Series) -> float:
        n_samples, _ = X.shape
        grad = 2 / n_samples * (X.T @ (X @ self.weights - y))
        
        if self.reg:
            if self.reg == 'l1' or self.reg == 'elasticnet':
                grad += self.l1_coef * np.sign(self.weights)
            if self.reg == 'l2' or self.reg == 'elasticnet':
                grad += self.l2_coef * 2 * self.weights
        
        return grad
    
    def get_coef(self) -> pd.Series:
        return self.weights[1:]

    def get_best_score(self) -> float:
        return self.best_score
        
    def __str__(self) -> str:
        return f"MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}" + \
            f' metric={self.metric}, reg={self.reg}, sgd_sample={self.sgd_sample}'
    
    def __repr__(self) -> str:
        return f'MyLineReg class: {self.__dict__}'
    


In [181]:
LineReg = MyLineReg(n_iter = 100, 
              learning_rate = lambda iter: 0.7 * (0.9 ** iter), 
              metric = 'mse', 
              reg = 'l1', 
              l1_coef = 0.1, 
              l2_coef = 0.0)

LineReg.fit(X_train, y_train)
y_pred = LineReg.predict(X_test)
LineReg.get_best_score()

226.73336547132743

In [182]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
y_pred = LinearRegression().fit(X_train, y_train).predict(X_test)
mean_squared_error(y_test, y_pred)

223.41974177996778